# SD212: Graph mining

## Lab 7: Graph neural networks

In this lab, you will learn to classify nodes using graph neural networks.

We use [DGL](https://www.dgl.ai) (deep graph library), which relies on pytorch.

In [1]:
# !pip install dgl

## Import

In [2]:
import warnings

import dgl
import numpy as np
import torch
import torch.nn.functional as F
from dgl import function as fn
from dgl.nn import SAGEConv
from scipy import sparse
from sknetwork.classification import DiffusionClassifier
from sknetwork.data import load_netset
from sknetwork.embedding import Spectral
from sknetwork.utils import directed2undirected
from torch import nn

warnings.filterwarnings("ignore")

## Load data

We will work on the following datasets (see the [NetSet](https://netset.telecom-paris.fr/) collection for details):
* Cora (directed graph + bipartite graph)
* WikiVitals (directed graph + bipartite graph)

Both datasets are graphs with node features (given by the bipartite graph) and ground-truth labels.

In [3]:
cora = load_netset("cora")
wikivitals = load_netset("wikivitals")

Parsing files...
Done.
Parsing files...
Done.


In [4]:
dataset = cora
# dataset = wikivitals

In [5]:
adjacency = dataset.adjacency
biadjacency = dataset.biadjacency
labels = dataset.labels

In [6]:
# we use undirected graphs
adjacency = directed2undirected(adjacency)

In [7]:
# for Wikivitals, use spectral embedding of the bipartite graph as features

if dataset.meta.name.startswith("Wikivitals"):
    spectral = Spectral(50)
    features = spectral.fit_transform(biadjacency)
else:
    features = biadjacency.toarray()

In [8]:
def split_train_test_val(n_samples, test_ratio=0.1, val_ratio=0.1, seed=None):
    """Split the samples into train / test / validation sets.

    Returns
    -------
    train: np.ndarray
        Boolean mask
    test: np.ndarray
        Boolean mask
    validation: np.ndarray
        Boolean mask
    """
    if seed:
        np.random.seed(seed)

    # test
    index = np.random.choice(
        n_samples, int(np.ceil(n_samples * test_ratio)), replace=False
    )
    test = np.zeros(n_samples, dtype=bool)
    test[index] = 1

    # validation
    index = np.random.choice(
        np.argwhere(~test).ravel(), int(np.ceil(n_samples * val_ratio)), replace=False
    )
    val = np.zeros(n_samples, dtype=bool)
    val[index] = 1

    # train
    train = np.ones(n_samples, dtype=bool)
    train[test] = 0
    train[val] = 0
    return train, test, val

In [9]:
train, test, val = split_train_test_val(len(labels))

## Graph and tensors

In DGL, the graph is represented as an object, the features and labels as tensors.

In [10]:
# graph as an object
graph = dgl.from_scipy(adjacency)

In [11]:
type(graph)

dgl.heterograph.DGLGraph

In [12]:
# features and labels as tensors
features = torch.Tensor(features)
labels = torch.Tensor(labels).long()

In [13]:
# masks as tensors
train = torch.Tensor(train).bool()
test = torch.Tensor(test).bool()
val = torch.Tensor(val).bool()

## Graph neural network

We start with a simple graph neural network without hidden layer. The output layer is of type [GraphSAGE](https://docs.dgl.ai/generated/dgl.nn.pytorch.conv.SAGEConv.html).

In [14]:
class GNN(nn.Module):
    def __init__(self, dim_input, dim_output):
        super(GNN, self).__init__()
        self.conv = SAGEConv(dim_input, dim_output, aggregator_type="mean")

    def forward(self, graph, features):
        output = self.conv(graph, features)
        return output

## To do

* Train the model on Cora and get accuracy.
* Compare with the same model trained on an empty graph.
* Add a hidden layer with ReLu activation function (e.g., dimension = 20) and retrain the model. 
* Compare with a classifier based on heat diffusion.

In [15]:
def init_model(model, features, labels):
    """Init the GNN with appropriate dimensions."""
    dim_input = features.shape[1]
    dim_output = len(labels.unique())
    return model(dim_input, dim_output)

In [16]:
def eval_model(gnn, graph, features, labels, test=test):
    """Evaluate the model in terms of accuracy."""
    gnn.eval()
    with torch.no_grad():
        output = gnn(graph, features)
        labels_pred = torch.max(output, dim=1)[1]
        score = np.mean(np.array(labels[test]) == np.array(labels_pred[test]))
    return score

In [17]:
def train_model(
    gnn,
    graph,
    features,
    labels,
    train=train,
    val=val,
    n_epochs=100,
    lr=0.01,
    verbose=True,
):
    """Train the GNN."""
    optimizer = torch.optim.Adam(gnn.parameters(), lr=lr)

    gnn.train()
    scores = []

    for t in range(n_epochs):

        # forward
        output = gnn(graph, features)
        logp = nn.functional.log_softmax(output, 1)
        loss = nn.functional.nll_loss(logp[train], labels[train])

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # evaluation
        score = eval_model(gnn, graph, features, labels, val)
        scores.append(score)

        if verbose and t % 10 == 0:
            print(
                "Epoch {:02d} | Loss {:.3f} | Accuracy {:.3f}".format(
                    t, loss.item(), score
                )
            )

In [18]:
gnn = init_model(GNN, features, labels)

In [19]:
train_model(gnn, graph, features, labels)

Epoch 00 | Loss 1.938 | Accuracy 0.387
Epoch 10 | Loss 0.585 | Accuracy 0.827
Epoch 20 | Loss 0.247 | Accuracy 0.852
Epoch 30 | Loss 0.135 | Accuracy 0.856
Epoch 40 | Loss 0.087 | Accuracy 0.860
Epoch 50 | Loss 0.063 | Accuracy 0.852
Epoch 60 | Loss 0.050 | Accuracy 0.841
Epoch 70 | Loss 0.041 | Accuracy 0.834
Epoch 80 | Loss 0.035 | Accuracy 0.834
Epoch 90 | Loss 0.030 | Accuracy 0.834


In [20]:
eval_model(gnn, graph, features, labels)

0.8450184501845018

In [21]:
empty_graph = dgl.from_scipy(sparse.csr_matrix(adjacency.shape))

In [22]:
gnn = init_model(GNN, features, labels)

In [23]:
train_model(gnn, graph, features, labels)

Epoch 00 | Loss 1.984 | Accuracy 0.347
Epoch 10 | Loss 0.601 | Accuracy 0.823
Epoch 20 | Loss 0.252 | Accuracy 0.852
Epoch 30 | Loss 0.137 | Accuracy 0.852
Epoch 40 | Loss 0.088 | Accuracy 0.845
Epoch 50 | Loss 0.064 | Accuracy 0.841
Epoch 60 | Loss 0.050 | Accuracy 0.841
Epoch 70 | Loss 0.042 | Accuracy 0.834
Epoch 80 | Loss 0.035 | Accuracy 0.827
Epoch 90 | Loss 0.031 | Accuracy 0.827


In [24]:
eval_model(gnn, empty_graph, features, labels, test)

0.6568265682656826

In [25]:
class GNN(nn.Module):
    def __init__(self, dim_input, dim_output, dim_hidden=20):
        super(GNN, self).__init__()
        self.conv1 = SAGEConv(dim_input, dim_hidden, "mean")
        self.conv2 = SAGEConv(dim_hidden, dim_output, "mean")

    def forward(self, graph, features):
        h = self.conv1(graph, features)
        h = F.relu(h)
        output = self.conv2(graph, h)
        return output

In [26]:
gnn = init_model(GNN, features, labels)

In [27]:
train_model(gnn, graph, features, labels)

Epoch 00 | Loss 1.982 | Accuracy 0.454
Epoch 10 | Loss 0.163 | Accuracy 0.863
Epoch 20 | Loss 0.025 | Accuracy 0.867
Epoch 30 | Loss 0.007 | Accuracy 0.867
Epoch 40 | Loss 0.003 | Accuracy 0.867
Epoch 50 | Loss 0.002 | Accuracy 0.863
Epoch 60 | Loss 0.001 | Accuracy 0.867
Epoch 70 | Loss 0.001 | Accuracy 0.863
Epoch 80 | Loss 0.001 | Accuracy 0.863
Epoch 90 | Loss 0.001 | Accuracy 0.863


In [28]:
eval_model(gnn, graph, features, labels)

0.8597785977859779

In [29]:
# comparison with heat diffusion
y_true = dataset.labels
y_train = y_true.copy()
y_train[test | val] = -1

In [30]:
algo = DiffusionClassifier()

In [31]:
y_pred = algo.fit_predict(biadjacency, y_train)

In [32]:
np.mean(y_pred[test] == y_true[test])

0.7859778597785978

In [33]:
y_pred = algo.fit_predict(adjacency, y_train)

In [34]:
np.mean(y_pred[test] == y_true[test])

0.8597785977859779

## Build your own GNN

You will now build your own GNN. We start with a simple graph convolution layer. 

In [35]:
class GraphConvLayer(nn.Module):
    def __init__(self, dim_input, dim_output):
        super(GraphConvLayer, self).__init__()
        self.layer = nn.Linear(dim_input, dim_output)

    def forward(self, graph, signal):
        with graph.local_scope():
            # message passing
            graph.ndata["node"] = signal
            graph.update_all(
                fn.copy_u("node", "message"), fn.mean("message", "average")
            )
            h = graph.ndata["average"]
            return self.layer(h)

Observe that the message passing is based on the diffusion:
$$U\mapsto PU$$ where $P$ is the transition matrix of the random walk.

## To do

* Build a GNN with two layers based on this graph convolution layer.
* Train this GNN and compare the results with the previous one.
* Add the input signal of each node, so that the message passing becomes:
$$
U\mapsto (I + P)U
$$
* Retrain the GNN and observe the results.
* Retrain the same GNN without message passing in the first layer.

In [36]:
class GNN(nn.Module):
    def __init__(self, n_input, n_output, n_hidden=20):
        super(GNN, self).__init__()
        self.conv1 = GraphConvLayer(n_input, n_hidden)
        self.conv2 = GraphConvLayer(n_hidden, n_output)

    def forward(self, graph, features):
        h = self.conv1(graph, features)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [37]:
gnn = init_model(GNN, features, labels)
train_model(gnn, graph, features, labels, train, val, verbose=True)

Epoch 00 | Loss 1.958 | Accuracy 0.310
Epoch 10 | Loss 1.227 | Accuracy 0.697
Epoch 20 | Loss 0.519 | Accuracy 0.860
Epoch 30 | Loss 0.283 | Accuracy 0.875
Epoch 40 | Loss 0.201 | Accuracy 0.867
Epoch 50 | Loss 0.153 | Accuracy 0.841
Epoch 60 | Loss 0.122 | Accuracy 0.834
Epoch 70 | Loss 0.100 | Accuracy 0.830
Epoch 80 | Loss 0.083 | Accuracy 0.838
Epoch 90 | Loss 0.070 | Accuracy 0.838


In [38]:
eval_model(gnn, graph, features, labels, test)

0.8302583025830258

In [39]:
class GraphConvLayer(nn.Module):
    def __init__(self, dim_input, dim_output):
        super(GraphConvLayer, self).__init__()
        self.layer = nn.Linear(dim_input, dim_output)

    def forward(self, graph, signal):
        with graph.local_scope():
            # message passing
            graph.ndata["node"] = signal
            graph.update_all(
                fn.copy_u("node", "message"), fn.mean("message", "average")
            )
            h = graph.ndata["average"]
            return self.layer(h + signal)

In [40]:
class GNN(nn.Module):
    def __init__(self, n_input, n_output, n_hidden=20):
        super(GNN, self).__init__()
        self.conv1 = GraphConvLayer(n_input, n_hidden)
        self.conv2 = GraphConvLayer(n_hidden, n_output)

    def forward(self, graph, features):
        h = self.conv1(graph, features)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [41]:
gnn = init_model(GNN, features, labels)
train_model(gnn, graph, features, labels, train, val, verbose=True)

Epoch 00 | Loss 1.937 | Accuracy 0.483
Epoch 10 | Loss 0.307 | Accuracy 0.871
Epoch 20 | Loss 0.132 | Accuracy 0.875
Epoch 30 | Loss 0.068 | Accuracy 0.867
Epoch 40 | Loss 0.042 | Accuracy 0.860
Epoch 50 | Loss 0.029 | Accuracy 0.863
Epoch 60 | Loss 0.022 | Accuracy 0.867
Epoch 70 | Loss 0.017 | Accuracy 0.860
Epoch 80 | Loss 0.015 | Accuracy 0.860
Epoch 90 | Loss 0.013 | Accuracy 0.860


In [42]:
eval_model(gnn, graph, features, labels, test)

0.8708487084870848

In [43]:
class ConvLayer(nn.Module):
    def __init__(self, dim_input, dim_output):
        super(ConvLayer, self).__init__()
        self.layer = nn.Linear(dim_input, dim_output)

    def forward(self, features):
        output = self.layer(features)
        return output

In [44]:
class GNN(nn.Module):
    def __init__(self, n_input, n_output, n_hidden=20):
        super(GNN, self).__init__()
        self.conv1 = ConvLayer(n_input, n_hidden)
        self.conv2 = GraphConvLayer(n_hidden, n_output)

    def forward(self, graph, features):
        h = self.conv1(features)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [45]:
gnn = init_model(GNN, features, labels)

In [46]:
train_model(gnn, graph, features, labels, train, val, verbose=True)

Epoch 00 | Loss 1.912 | Accuracy 0.328
Epoch 10 | Loss 0.550 | Accuracy 0.856
Epoch 20 | Loss 0.153 | Accuracy 0.875
Epoch 30 | Loss 0.060 | Accuracy 0.889
Epoch 40 | Loss 0.031 | Accuracy 0.886
Epoch 50 | Loss 0.019 | Accuracy 0.878
Epoch 60 | Loss 0.014 | Accuracy 0.875
Epoch 70 | Loss 0.011 | Accuracy 0.875
Epoch 80 | Loss 0.009 | Accuracy 0.878
Epoch 90 | Loss 0.008 | Accuracy 0.878


In [47]:
eval_model(gnn, graph, features, labels, test)

0.8819188191881919

## Heat diffusion as a GNN

Node classification by heat diffusion can be seen as a GNN without training, using a one-hot encoding of labels. Features are ignored.

## To do

* Build a special GNN whose output corresponds to one step of heat diffusion in the graph.
* Use this GNN to classify nodes by heat diffusion, with temperature centering.

In [48]:
from sknetwork.utils import get_membership

In [49]:
labels_one_hot = get_membership(labels).toarray()
labels_one_hot = torch.Tensor(labels_one_hot)

In [50]:
class Diffusion(nn.Module):
    def __init__(self):
        super(Diffusion, self).__init__()

    def forward(self, graph, features, mask):
        """Mask is a boolean tensor giving the training set."""
        with graph.local_scope():
            h_node = features
            # diffusion
            graph.ndata["node"] = h_node
            graph.update_all(
                fn.copy_src("node", "message"), fn.mean("message", "neighbor")
            )
            h_neighbor = graph.ndata["neighbor"]
            # seed nodes
            h_neighbor[mask] = h_node[mask]
            return h_neighbor

In [51]:
diffusion = Diffusion()

In [52]:
n_iter = 20

temperatures = labels_one_hot
temperatures[~train] = 0
for t in range(n_iter):
    temperatures = diffusion(graph, temperatures, train)

# temperature centering
temperatures -= temperatures.mean(axis=0)

AttributeError: module 'dgl.function' has no attribute 'copy_src'

In [ ]:
labels_pred = np.argmax(temperatures.numpy(), axis=1)

In [ ]:
np.mean(labels.numpy()[test] == labels_pred[test])

In [ ]:
# comparison with scikit-network
from sknetwork.classification import DiffusionClassifier

In [ ]:
algo = DiffusionClassifier()

In [ ]:
labels[~train] = -1
labels = labels.numpy()
labels_true = dataset.labels

In [ ]:
labels_pred = algo.fit_predict(adjacency, labels)

In [ ]:
np.mean(labels_true[test] == labels_pred[test])